# Preprocessing of Social Care Data Landscape sources for input to the visualisation
This python3 notebook preprocesses the source data to create a single csv file for the visualisation.
Refer to the [data model](../data_model/scdl_data_model_img.png) for orientation.

In [31]:
import pandas as pd
import csv

## Load source data tables

In [38]:
df_dm = pd.read_csv('../data/domain_v1.31.csv')                     # THF domain model for adult social care.
df_msr = pd.read_csv('../data/measure_v1.3.csv')                    # Measures from publicly available sources.
df_dm_msr_map = pd.read_csv('../data/domain_measure_map_v1.3.csv')  # Mapping from domain model to measures.

print(f'number of domain model nodes = {len(df_dm)}')
print(f'number of publicly available measures = {len(df_msr)}')
print(f'number of mappings = {len(df_dm_msr_map)}')

number of domain model nodes = 100
number of publicly available measures = 550
number of mappings = 1512


## Join the tables
An outer join from the domain model to the publicly available measures preserves the full domain model. This means we can call out the gaps where no measures exist.

In [39]:
# remove logically deleted maps to duplicated measures
df_dm_msr_map = df_dm_msr_map[df_dm_msr_map.logical_delete!='Y']
df_dm_msr_map.drop('logical_delete',axis=1,inplace=True)
print(f'number of maps after logical delete removal = {len(df_dm_msr_map)}')

# inner join from map to measures
df_map_msr = pd.merge(df_dm_msr_map, df_msr, on='msr_id')
print(f'length of inner join from map to measure, df_map_msr = {len(df_map_msr)}')

# outer join from domain model to measures
df = pd.merge(df_dm, df_map_msr, on='dm_id', how='left')
print(f'length of outer join from domain model to measure mappings, df = {len(df)}')
print(f'{len(df.msr_id.unique())} unique measures')

number of maps after logical delete removal = 1328
length of inner join from map to measure, df_map_msr = 1328
length of outer join from domain model to measure mappings, df = 1338
352 unique measures


In [40]:
# Take a peek
df.head(5)

,dm_id,dm_l1_name,dm_l2_name,dm_l3_name,dm_seq,dm_sort_l1,dm_sort_l2,dm_sort_l3,dm_phase,msr_id,...,src_year_first_published,src_year_last_published,src_discontinuity,src_pub_frequency,src_pub_lag_months,src_data_provider,src_statistic_class,src_methodology_url,src_quality_url,msr_strength
0,1,Funders,Budget,Local authorities,1.1.01,1,1,1,supply,328.0,...,2008.0,2019.0,0,Annual,TBD,"Ministry of Housing, Communities & Local Gover...",National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89
1,1,Funders,Budget,Local authorities,1.1.01,1,1,1,supply,329.0,...,2008.0,2019.0,0,Annual,TBD,"Ministry of Housing, Communities & Local Gover...",National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89
2,1,Funders,Budget,Local authorities,1.1.01,1,1,1,supply,330.0,...,2008.0,2019.0,0,Annual,TBD,"Ministry of Housing, Communities & Local Gover...",National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89
3,1,Funders,Budget,Local authorities,1.1.01,1,1,1,supply,331.0,...,2008.0,2019.0,0,Annual,TBD,"Ministry of Housing, Communities & Local Gover...",National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89
4,1,Funders,Budget,Local authorities,1.1.01,1,1,1,supply,332.0,...,2008.0,2019.0,0,Annual,TBD,"Ministry of Housing, Communities & Local Gover...",National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89


In [41]:
# Make sure the outer join has preserved un-mapped domain model nodes (in this case investors)
df[df.dm_id==9]

,dm_id,dm_l1_name,dm_l2_name,dm_l3_name,dm_seq,dm_sort_l1,dm_sort_l2,dm_sort_l3,dm_phase,msr_id,...,src_year_first_published,src_year_last_published,src_discontinuity,src_pub_frequency,src_pub_lag_months,src_data_provider,src_statistic_class,src_methodology_url,src_quality_url,msr_strength
139,9,Providers,Investors,Investors,2.1.01,2,4,9,supply,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create sort fields to preserve order of domain model nodes at each chart level

In [42]:
# sort into domain model sequence order
df['dm_func_seq'] = df.dm_seq
df = df.sort_values(['dm_seq','dm_phase', 'msr_id'])
df.dm_seq = df.index.values # create unique sequence per row

# create composite domain node names at levels 2 and 3
df['dm_l1_l2_name'] = df['dm_l1_name'] + ' : ' + df['dm_l2_name']
df['dm_l1_l3_name'] = df['dm_l1_l2_name'] + ' : ' + df['dm_l3_name']

# Avoid interleaving of sort values at each level.
df ['dm_sort_l1'] = df.dm_sort_l1 + 1000
df ['dm_sort_l2'] = df.dm_sort_l2 + 2000
df ['dm_sort_l3'] = df.dm_sort_l3 + 3000

## Create measure strengths for each chart level
This supports sort by measure strength in the visualisation

In [43]:
# Create measure strength for level 1
df['msr_strength_l1_fix'] = 0.0
df = df.sort_values(['msr_id','dm_l1_name','dm_phase'])
last_msrid_l1_phase = ''
for i, _ in df.iterrows():

    if pd.isnull(df.at[i,'msr_id']):
        continue

    msrid_l1_phase = df.at[i,'msr_id'].astype('str') + df.at[i,'dm_l1_name'] \
                     + df.at[i,'dm_phase']

    if msrid_l1_phase != last_msrid_l1_phase:
        df.at[i,'msr_strength_l1_fix'] = df.at[i,'msr_strength']

    last_msrid_l1_phase = msrid_l1_phase

# Create measure strength for level 2
df['msr_strength_l2_fix'] = 0.0
df = df.sort_values(['msr_id','dm_l1_name','dm_l2_name','dm_phase'])
last_msrid_l1_l2_phase = ''
for i, _ in df.iterrows():

    if pd.isnull(df.at[i,'msr_id']):
        continue

    msrid_l1_l2_phase = df.at[i,'msr_id'].astype('str') + df.at[i,'dm_l1_name'] \
                        + df.at[i,'dm_l2_name'] + df.at[i,'dm_phase']

    if msrid_l1_l2_phase != last_msrid_l1_l2_phase:
        df.at[i,'msr_strength_l2_fix'] = df.at[i,'msr_strength']

    last_msrid_l1_l2_phase = msrid_l1_l2_phase

# Reinstate the sort
df = df.sort_values(['dm_seq','dm_phase', 'msr_id'])

## Create sort by phase
Enables the visualisation to sort the measures by phase (demand, supply, operate, outcome), using a bitmap of those four variables.

In [44]:
# Create sort by for phase bitmap.
df_pivot = df[['msr_id','dm_phase','dm_seq']]
df_pivot = df_pivot.pivot_table(index='msr_id', columns='dm_phase', aggfunc=lambda x: 1, fill_value=0)
df_pivot.columns = df_pivot.columns.levels[1]
df_pivot.reset_index()
df_pivot['phase_bitmap'] = df_pivot.outcome.astype(str)+\
                           df_pivot.operate.astype(str)+\
                           df_pivot.supply.astype(str)+\
                           df_pivot.demand.astype(str)
df_pivot.drop(['demand','operate','outcome','supply'], axis=1, inplace=True)
df = df.merge(df_pivot, how='left', on='msr_id')

In [45]:
# Take a final peek
df.head(5)

,dm_id,dm_l1_name,dm_l2_name,dm_l3_name,dm_seq,dm_sort_l1,dm_sort_l2,dm_sort_l3,dm_phase,msr_id,...,src_statistic_class,src_methodology_url,src_quality_url,msr_strength,dm_func_seq,dm_l1_l2_name,dm_l1_l3_name,msr_strength_l1_fix,msr_strength_l2_fix,phase_bitmap
0,1,Funders,Budget,Local authorities,0,1001,2001,3001,supply,328.0,...,National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89,1.1.01,Funders : Budget,Funders : Budget : Local authorities,0.89,0.89,0011
1,1,Funders,Budget,Local authorities,1,1001,2001,3001,supply,329.0,...,National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89,1.1.01,Funders : Budget,Funders : Budget : Local authorities,0.89,0.89,0011
2,1,Funders,Budget,Local authorities,2,1001,2001,3001,supply,330.0,...,National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89,1.1.01,Funders : Budget,Funders : Budget : Local authorities,0.89,0.89,0011
3,1,Funders,Budget,Local authorities,3,1001,2001,3001,supply,331.0,...,National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89,1.1.01,Funders : Budget,Funders : Budget : Local authorities,0.89,0.89,0011
4,1,Funders,Budget,Local authorities,4,1001,2001,3001,supply,332.0,...,National,https://assets.publishing.service.gov.uk/gover...,https://assets.publishing.service.gov.uk/gover...,0.89,1.1.01,Funders : Budget,Funders : Budget : Local authorities,0.89,0.89,0011


## Export the data

In [46]:
df = df.sort_values(['dm_seq','dm_phase', 'msr_id'])
df.to_csv('../data/dm_map_msr_joined_v1.31.csv',
          index=False, 
          quoting=csv.QUOTE_NONNUMERIC)